# Project - Neural Networks using Keras

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since have a relatively small data set, I'm using cross-validation to evaluate the accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Performing data prep here. One can use data pipelines. Eventually, one can do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [6]:
def new_col(df):
    
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()

    # Use the formula, though fill in 0s when the value is 0/0 (because 0/0 generates "nan" values)
    df1['ratio_D_C'] = (df1['DELINQ']/df1['CLNO']).fillna(0)

    # Replace the infinity values with 1 (because a value divided by 0 generates infinity)
    df1['ratio_D_C'].replace(np.inf, 1, inplace=True)

    return df1[['ratio_D_C']]
    # Checking whether the calculation is made correctly:
    #return df1
    

In [7]:
#Let's test the new function:

# Send the train set to the function we created
new_col(train)


,ratio_D_C
1552,0.000000
2290,0.000000
1398,0.000000
1775,0.035714
2299,0.000000
...,...
1638,0.125000
1095,0.181818
1130,0.000000
1294,0.000000


##  Identify the numeric, binary, and categorical columns

In [8]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [9]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [10]:
categorical_columns

['REASON', 'JOB']

In [11]:
feat_eng_columns = ['DELINQ','CLNO']

# Pipeline

In [12]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [13]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [14]:
# Create a pipeline for the transformed column here
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])



In [17]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])


In [1]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
    
  #      ENTER THE TRANSFORMER FOR THE NEW COLUMN HERE
        ('trans', my_new_column, feat_eng_columns)
        ],   
        remainder='passthrough')

#passtrough is an optional step.

NameError: name 'ColumnTransformer' is not defined

# Transform: fit_transform() for TRAIN

In [19]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        , -0.42345858],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        , -0.42345858],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        , -0.42345858],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        , -0.42345858],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        , -0.42345858],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        , -0.42345858]])

In [20]:
train_x.shape

(1750, 21)

# Tranform: transform() for TEST

In [21]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        , -0.42345858],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        , -0.42345858],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        , -0.42345858],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  2.31481716],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        , -0.42345858],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        , -0.42345858]])

In [22]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [23]:
# Find percentage
train_target.value_counts()/len(train_target)

BAD
0    0.603429
1    0.396571
Name: count, dtype: float64

# Train a shallow (one-layer) Keras model

In [27]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [28]:
#What is the input shape?
#(meaning: how many neurons should be in the input layer?)

train_x.shape

(1750, 21)

In [31]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=21))
model.add(keras.layers.Dense(18, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

#final layer: there has to be 1 node with softmax


In [34]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [37]:
# Fit the model

history = model.fit(train_x, train_target, 
                    validation_data=(test_x, test_target), 
                    epochs=20, batch_size=100)

Epoch 1/20


18/18 [==============================] - 3s 37ms/step - loss: 0.6320 - accuracy: 0.6554 - val_loss: 0.5592 - val_accuracy: 0.7347
Epoch 2/20
18/18 [==============================] - 0s 14ms/step - loss: 0.5233 - accuracy: 0.7480 - val_loss: 0.5323 - val_accuracy: 0.7427
Epoch 3/20
18/18 [==============================] - 0s 16ms/step - loss: 0.4999 - accuracy: 0.7520 - val_loss: 0.5153 - val_accuracy: 0.7427
Epoch 4/20
18/18 [==============================] - 0s 11ms/step - loss: 0.4886 - accuracy: 0.7606 - val_loss: 0.5121 - val_accuracy: 0.7347
Epoch 5/20
18/18 [==============================] - 0s 12ms/step - loss: 0.4802 - accuracy: 0.7600 - val_loss: 0.5004 - val_accuracy: 0.7413
Epoch 6/20
18/18 [==============================] - 0s 7ms/step - loss: 0.4715 - accuracy: 0.7737 - val_loss: 0.4902 - val_accuracy: 0.7520
Epoch 7/20
18/18 [==============================] - 0s 6ms/step - loss: 0.4645 - accuracy: 0.7749 - val_loss: 0.4952 - val_accuracy: 0.7453
Epoch 8/20
18

In [39]:
# Train values

train_scores = model.evaluate(train_x, train_target, verbose=0)

train_scores

# In results, first is loss, second is accuracy

[0.3863182067871094, 0.8217142820358276]

In [40]:
# Print the values

print(f"Train {model.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {model.metrics_names[1]}: {train_scores[1]*100:.2f}%")


Train loss: 0.39
Train accuracy: 82.17%


In [41]:
# Test values

test_scores = model.evaluate(test_x, test_target, verbose=0)

test_scores

# In results, first is loss, second is accuracy

[0.4504648447036743, 0.7853333353996277]

In [42]:
# Print the values

print(f"Test {model.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {model.metrics_names[1]}: {test_scores[1]*100:.2f}%")


Test loss: 0.45
Test accuracy: 78.53%


# Train a deep (multi-layered) Keras model 

In [43]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=21))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

#final layer: there has to be 4 nodes with softmax (because we have 4 categories)

In [44]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [46]:
# Fit the model

history = model.fit(train_x, train_target, 
                    validation_data=(test_x, test_target), 
                    epochs=20, batch_size=100)

Epoch 1/20
18/18 [==============================] - 2s 28ms/step - loss: 0.5857 - accuracy: 0.6903 - val_loss: 0.5366 - val_accuracy: 0.7453
Epoch 2/20
18/18 [==============================] - 0s 9ms/step - loss: 0.4966 - accuracy: 0.7737 - val_loss: 0.5132 - val_accuracy: 0.7320
Epoch 3/20
18/18 [==============================] - 0s 9ms/step - loss: 0.4749 - accuracy: 0.7794 - val_loss: 0.4913 - val_accuracy: 0.7547
Epoch 4/20
18/18 [==============================] - 0s 11ms/step - loss: 0.4540 - accuracy: 0.7897 - val_loss: 0.4965 - val_accuracy: 0.7573
Epoch 5/20
18/18 [==============================] - 0s 8ms/step - loss: 0.4464 - accuracy: 0.7971 - val_loss: 0.4949 - val_accuracy: 0.7667
Epoch 6/20
18/18 [==============================] - 0s 8ms/step - loss: 0.4283 - accuracy: 0.8069 - val_loss: 0.4990 - val_accuracy: 0.7467
Epoch 7/20
18/18 [==============================] - 0s 9ms/step - loss: 0.4127 - accuracy: 0.8051 - val_loss: 0.4943 - val_accuracy: 0.7520
Epoch 8/20
18/18 [

In [47]:
# Train values

train_scores = model.evaluate(train_x, train_target, verbose=0)

train_scores

# In results, first is loss, second is accuracy

[0.20357029139995575, 0.9188571572303772]

In [48]:
# Print the values

print(f"Train {model.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {model.metrics_names[1]}: {train_scores[1]*100:.2f}%")


Train loss: 0.20
Train accuracy: 91.89%


In [50]:
# Test values

test_scores = model.evaluate(test_x, test_target, verbose=0)

test_scores

# In results, first is loss, second is accuracy

[0.49086812138557434, 0.8360000252723694]

In [51]:
# Print the values

print(f"Test {model.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {model.metrics_names[1]}: {test_scores[1]*100:.2f}%")


Test loss: 0.49
Test accuracy: 83.60%


# Optional: try different activation functions, optimizers, or configurations (such as wide and deep) to build other models